<a href="https://colab.research.google.com/github/irezcen/PUM_sieci_neuronowe/blob/CNN/Lab8_fully_connected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sieci neuronowe są bardzo rozbudowanym elementem uczenia maszynowego. Często pozwalaja uzyskać bardzo dobre rezultaty w sytuacji, gdy standardowe modele klasyfikacyjne są zbyt proste i nie radzą sobie z danym zagadnieniem.

Do implementacji sieci neuronowych w Pythonie stosowane są głównie dwa pakiety: TensorFlow i PyTorch. My będziemy używać tego drugiego.

Zarówno TensorFlow, jak i PyTorch, zapewniają bardzo dużą swobodę w definiowaniu architektury sieci. Jest to ich duża zaleta, jednak dla początkujących osób może być to trochę problematyczne - każda klasa musi być odpowiednio zdefiniowana, trzeba samodzielnie napisać funkcje, które będą wykorzystywane do wczytywania danych i każdej epoki treningu sieci itd. Żeby ułatwić pracę użytkownikom mniej zaawansowanym i ustrukturyzować kod, stworzone zostały nakładki na oba pakiety, które upraszczają pracę z sieciami neuronowymi. W przypadku PyTorcha najczęściej używane są dwie z nich: Ignite i Lightning. My będziemy używać tego pierwszego.

Do używania pakietu Ignite konieczna jest jego instalacja oraz wcześniejsza instalacja Pytorcha. Jeżeli jeszcze nie masz ich zainstalowanych, możesz zrobić to poniższym kodem. Jeżeli masz już zainstalowane oba pakiety, możesz usunąć tę komórkę i przejść od razu do importu bibliotek.

In [1]:
!pip install pytorch-ignite
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from ignite.engine import create_supervised_trainer, create_supervised_evaluator, Events
from ignite.metrics import Loss, Accuracy
from ignite.contrib.handlers import ProgressBar
from ignite.handlers import FastaiLRFinder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.1/264.1 KB 7.0 MB/s eta 0:00:00


Tematem, którym dotychczas nie zajmowaliśmy się na zajęciach, a stanowi bardzo istotną dziedzinę akustyki i uczenia maszynowego w akustyce jest rozpoznawanie mowy. Jest to też dobra tematyka do pokazania możliwości sieci neuronowych - już prosta sieć, wykorzystująca jedynie podstawowe rodzaje warstwo pozwala osiągnąć względnie dobre rezultaty.

Dzisiaj będziemy używać najbardziej podstawowego rodzaju sieci neuronowej - sieci typu feed-forward z warstwami typu fully connected.

Warstwy typu fully connected cechują się tym, że każdy neuron wejściowy warstwy połączony jest z każdym neuronem wyjściowym warstwy poprzedniej. Każde takie połączenie ma określoną wagę - te wagi to właśnie parametry sieci, które modyfikowane są podczas treningu sieci.

![caption](https://4.bp.blogspot.com/-eTAX5ZojPwE/WrOuZWgFBKI/AAAAAAABCsc/YXLYgbVT4NE-hf2SoCvhHFdH3ocps9cdgCLcBGAs/s320/Capture.PNG)

Sieć feed-forward cechuje się tym, że przepływ informacji pomiędzy warstwami jest jednokierunkowy i nie występują sprzężenia zwrotne - wyjście danej warstwy jest zawsze połączone tylko z wejściem kolejnej warstwy.

Do rozpoznawania mowy stosowane są często tzw. filterbank features. Są to cechy bardzo podobne do spektrogramu i wyznaczane są jako wartości energii sygnału w poszczególnych pasmach w kolejnych ramkach sygnału. Od spektrogramu różnią się tym, że pasma wyznaczane są przez określnone banki filtrów, czyli zbiory filtrów o zadanych parametrach. Najczęściej stosownanym bankiem filtrów jest bank filtrów melowych, który poznaliśmy przy okazji omawiania MFCC. Jak już mówiliśmy przy okazji tematu ekstrakcji cech, bank ten tworzony jest przez filtry trójkątne o szerokości zależnej od częstotliwości środkowej filtra:
![caption](https://www.researchgate.net/profile/Yusnita-Mohd-Ali/publication/288632263/figure/fig1/AS:613909065121800@1523378735077/Mel-filter-banks-basis-functions-using-20-Mel-filters-in-the-filter-bank.png)

Filterbank features mogą posłużyć do późniejszego wyznaczenia MFCC - wystarczy zastosować na nich transformację kosinusową.

Oba rodzaje cech znajdują zastosowanie w rozpoznawaniu mowy - poniżej przykład, jak różnią się one pomiędzy sobą.

![caption](https://haythamfayek.com/assets/posts/post1/filter_banks.jpg)

![caption](https://haythamfayek.com/assets/posts/post1/mfcc.jpg)

Źródło obrazów: https://haythamfayek.com/2016/04/21/speech-processing-for-machine-learning.html

Naszą pierwszą sieć służącą do rozpoznawania mowy będziemy uczyć używając cech filterbank. Przed ich ekstrakcją, przeprowadzona została filtracja filtrem preemfazy, który ma na celu uwydatnienie składowych o wyższych częstotliwościach. Dodatkowo uzyskane wartości przeliczono na skalę logarytmiczną. Wszystkie operacje zostały przeprowadzone przy użyciu biliboteki python_speech_features (http://python-speech-features.readthedocs.io/)

Dane, które będziemy wykorzystywać pochodzą z bazy TensorFlow Speech Commands v0.02 (http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz) i zawierają 35 słów:
- Yes,
- No,
- Up,
- Down,
- Left,
- Right,
- On,
- Off,
- Stop,
- Go,
- Zero,
- One,
- Two,
- Three,
- Four,
- Five,
- Six,
- Seven,
- Eight,
- Nine,
- Bed,
- Bird,
- Cat,
- Dog,
- Happy,
- House,
- Marvin,
- Sheila,
- Tree,
- Wow.

Sygnały mają długość 1s - wszystkie krótsze zostały symetrycznie uzupełnione zerami, by uzyskać stałą długość sygnału.

Żeby zredukować czas potrzebny na trening sieci oraz ograniczyć rozmiar danych, będziemy analizować tylko 300 losowo wybranych nagrań każdego ze słów. W praktyce powinno wykorzystywać się możliwe duży zbiór danych - możesz samodzielnie wyliczyć cechy ze wszystkich sygnałów w bazie i wykorzystać je do treningu sieci.

In [ ]:
feats = np.load('logfbank_feats.npy')
labels = np.load('labels.npy')

Macierz feats jest trójwymiarowa (liczba sygnałów x liczba ramek x liczba filtrów). Zwykła sieć typu feed-forward potrzebuje mieć dane opisujące pojedynczy obiekt podane w postaci wektora, a nie obrazu (macierzy 2D). Używamy więc funkcji reshape, żeby zmienić kształt danych na odpowiedni (liczba sygnałów x liczba cech, gdzie liczba cech=liczba ramek*liczba filtrów)

Dane podzielimy tym razem na trzy zbiory: uczący, walidacyjny i testowy. Zbiór uczący i testowy mają takie samo zastosowanie jak zawsze - uczący wykorzystywany jest do treningu modelu, a testowy do jego ewaluacji. Zbiór walidacyjny posłuży nam do ewaluacji modelu podczas treningu.

Dane podzielimy w taki sposób, by zbiór uczący zawierał 80% obiektów, a walidacyjny i testowy po 10%.

In [ ]:
feats = feats.reshape(feats.shape[0], -1)
feats = feats.astype(np.float32)

X_train, X_val_test, y_train, y_val_test = train_test_split(feats, labels, random_state=42, stratify=labels, train_size=0.8)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, random_state=42, stratify=y_val_test, train_size=0.5)

Pakiety do implementacji i uczenia sieci neuronowych wymagają, by dane były w postaci tensorów, a nie "zwykłych" macierzy, tak jak było w przypadku dotychczas poznawanych algorytmów. Musimy więc zamienić wektory z labelami oraz macierze z cechami na tensory i utworzyć z nich TensorDataset, czyli zbiór danych w formacie dostosowanym do użycia w sieci neuronowej. Zamianę na tensory robimy przy użyciu funkcji torch.tensor.

In [ ]:
trainset = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
valset = TensorDataset(torch.tensor(X_val), torch.tensor(y_val))
testset = TensorDataset(torch.tensor(X_test), torch.tensor(y_test))

TypeError: only size-1 arrays can be converted to Python scalars

Następnie tworzymy obiekty, które posłużą do wczytania danych przez sieć - robimy to przy pomocy funkcji DataLoader. Definiujemy w niej batch_size, czyli określamy, ile próbek ze zbioru będzie jednocześnie podane do sieci.
Podział zbioru na części (zwane wsadami, ang. batch) pozwala nie tylko zmniejszyć zużycie pamięci (co jest szczególnie istotne przy bardzo dużych zbiorach danych), ale też zazwyczaj przyspiesza proces uczenia.

Żeby w pełni zrozumieć, jak działa proces uczenia przy określonym podziale na wsady, trzeba znać pojęcie epoki.  Epoka to pojedyncze przejście wszystkich danych uczących przez całą architekturę sieci, podczas którego uaktualniane są wagi poszczególnych neuronów. Początkowo wagi dobrane są w sposób losowy (u nas) lub za pomocą aglorytmów (np. Kaiming, Xavier), a następnie zmieniane tak, by zminimalizować popełniane błędy (czyli zminimalizować loss lub zmaksymalizować dokładność).

Jeżeli mamy zdefiniowany batch_size mniejszy niż cały zbiór danych, to wagi aktualizowane są podczas uczenia na każdym wsadzie, jednak epoka kończy się dopiero po przejściu przez sieć danych zawartych we wszystkich wsadach. Epoka jest wtedy dzielona na iteracje, których liczba jest równa liczbie wsadów.

In [ ]:
train_loader = DataLoader(trainset, batch_size=256)
val_loader = DataLoader(valset, batch_size=256)
test_loader = DataLoader(testset, batch_size=256)

Po wczytaniu i odpowiednim przygotowaniu danych możemy przejść do definicji klasy, w której będzie zapisana architektura sieci.

W klasie definiujemy dwie funkcjie:
- init: w niej określamy, jakie warstwy będą występowały w sieci (ich rodzaj i rozmiar),
- forward: w niej określamy kolejność warstw, a tym samym kierunek przepływu danych wewnątrz sieci.

Tak jak wpominaliśmy wcześniej, dzisiaj będziemy używać jedynie warstw fully connected. Będą to zwykłe warstwy liniowe, które tworzy się używając klasy nn.Linear. Do klasy jako parametry podajemy wymiary warstwy: liczbę wejść oraz liczbę wyjść.

Liczba wejść warstwy musi być taka sama, jak liczba wyjść poprzedniej warstwy. W przypadku pierwszej warstwy w sieci liczba wejść musi być tak dobrana, by pasowała do rozmiaru danych uczących. W naszych danych każdy sygnał opisany jest przy pomocy 2574 cech, więc taka musi być liczba wejść pierwszej warstwy.

Liczba wyjść ostatniej warstwy powinna być równa liczbie klas, które znajdują się w danych. My mamy 35 klas, więc tyle też wyjść ustalamy w warstwie 3.

W funkcji forward określa się też, jaka funkcja aktywacji będzie użyta w ostatniej warstwie. Dobór funkcji aktywacji ma wpływ na uzyskiwane wyniki klasyfikacji, ponieważ to ona służy do obliczenia wartości pojawiającej się na wyjściu całej sieci. Jest wiele różnych funkcji aktywacji o różnych zastosowaniach - my będziemy używać funkcji log_softmax, czyli logarytmowanej znormalizowanej funkcji wykładniczej (zwanej też funkcją logistyczną). Softmax pozwala uzyskać na wyjściu sieci wektor prawdopodobieństw przynależności obiektu do każdej z klas, natomiast zastosowanie logarytmu jest szybsze pod względem numerycznym.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2574, 256)
        self.fc2 = nn.Linear(256, 120)
        self.fc3 = nn.Linear(120, 35)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

Musimy też określić, na czym będą wykonywane obliczenia, czyli czy mamy dostępną kartę graficzną, czy też będziemy korzystać ze zwykłego procesora. Robi się to używając zmiennej device - funkcją torch.cuda.is_available() sprawdzamy, czy jest dostępna karta graficzna. Jeżeli tak, to chcemy jej użyć (device="cuda"), jeżeli nie, to musimy zadowolić się procesorem (device="cpu").

Powoli zbliżamy się do zakończenia konfiguracji wszystkich ustawień sieci i treningu. Zostało nam do określenia jeszcze kilka istotnych parametrów. Przede wszystkim, musimy ustalić, jakie będzie kryterium uczenia się sieci i do czego będzie ona dążyć, czyli zmienną criterion. Użyjemy nn.NLLLoss (ang. negative log likelihood loss) - im mniejszą wartość będzie przyjmować, tym mniejsza różnica pomiędzy klasą rzeczywistą a predykcją sieci. NLLLoss jest odpowiednikiem entropii krzyżowej w sytuacji, gdy prawdopodobieństwa są zlogarytmowane (z użyciem log_softmax).

Mamy dużo klas, więc przeanalizujmy, jak jest to obliczane w prostszym przypadku softmax - entropia krzyżowa.

Przy okazji omawiania metryki log loss (straty logarytmicznej, inaczej nazywanej właśnie entropią krzyżową) wyliczaliśmy tzw. prawdopodobieństwa skorygowane. Wynikało to z tego, że binarne modele regresyjne mają tylko jedno wyjście i prawdopodobieństwo mniejsze od zdefiniowanego progu oznaczało przynależność do klasy 0, a większe - do klasy 1.

W przypadku sieci neuronowej i klasyfikacji wieloklasowej mamy tyle wyjść, ile klas, nie trzeba więc liczyć prawdopodobieństw skorygowanych - każde wyjście zwraca prawdopodobieństwo przynależności obiektu do jednej, konkretnej klasy. W takim przypadku entropia krzyżowa jest liczona jako ujemna średnia logarytmiczna prawdopodobieństw zwróconych przez odpowiednie wyjścia (te, które odpowiadają rzeczywistym klasom rozważanych obiektów).

Kolejnym istotnym krokiem jest dobór optimizera. Optimizer służy do optymalizacji wag modelu czyli zmieniania ich w taki sposób, żeby osiągnąć zadane kryterium, np. minimalną wartość NLLLoss.

Jednym z najczęściej używanych optimizerów jest optimizer SGD, który wykorzystuje stochastyczny spadek gradientu (ang. stochastic gradient descent). Metoda ta polega na wyliczaniu gradientu funkcji kosztu - gradient jest uogólnieniem pochodnej w sytuacji, gdy dotyczy ona wektora i zawiera wszystkie pochodne cząstkowe tego wektora. Gdy zbliżamy się do minimum funkcji, gradient zanika.

Prościej będzie wytłumaczyć zasadę działania bardziej podstawowej wersji algorytmu, czyli metody gradientu prostego, która składa się z 5 kroków:
1. wyznaczenie gradientu funkcji kosztu,
2. wybranie losowych wartości parametrów (w tym przypadku wag neuronów),
3. wyliczenie gradientu na podstawie wartości parametrów,
4. wyliczenie wielkości kroku, o który zmienimy wartości parametrów zgodnie ze wzorem: step size = gradient * learning rate
5. wyliczenie nowych wartości parametrów: new params = old params - step size,
6. powtarzenie kroków 3-5 do momentu uzyskania gradientu równego 0.

Jak już zostało wspomniane powyżej, wielkość kroków podczas zmiany parametrów określana jest przez współczynnik uczenia (ang. learning rate). Im większy współczynnik, tym większe większe kroki są stosowane - to zapewnia szybszy proces optymalizacji wag (a w konsekwencji uczenia sieci), ale też powoduje ryzyko "przeskoczenia" nad wartościa optymalną i może pogorszyć ostateczne rezultaty.
![caption](https://miro.medium.com/max/724/1*AqatzLelQw8LO9XuPCdfFw.png)

Z kolei zbyt mała wartość współczynnika znacząco wydłuża proces optymalizacji, co również nie jest korzystne.

Metoda SGD różni się od metody gradientu prostego tym, że nie korzysta się w niej z gradientu wyliczonego na całym zbiorze danych uczących, ponieważ jest to czaso- i kosztochłonne. Zamiast tego estymuje się wartość gradientu na podstawie losowo wybranego jednego elementu ze zbioru danych - stąd nazwa stochastyczny gradient.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# print(device)
criterion = nn.NLLLoss()
model = Net()
model.to(device)  # Move model before creating optimizer
optimizer = optim.SGD(model.parameters(), lr=1e-8, momentum=0.9)

W kolejnym kroku inicjalizujemy zmienne, w których będą zapisywane aktualne stany modelu i optimizera.

In [ ]:
init_model_state = model.state_dict()
init_opt_state = optimizer.state_dict()

Następnie musimy zdefiniować dwa obiekty - trainer i evaluator.

Trainer służy do uczenia sieci - podaje się do niego model (sieć), criterion i device. Podczas użycia trainera aktualizowane są wagi modelu w taki sposób, by dążyć do ekstremum zdefiniowanego kryterium (w tym przypadku do minimum, bo naszym kryterium jest NLLLoss).

Evaluator służy do ewaluacji sieci, czyli do walidacji oraz do wyliczenia metryk na zbiorze testowym - podczas jego użycia nie są aktualizowane wagi. Do niego podajemy model oraz metryki, które mają być użyte.

Dodatkowo, żeby mieć kontrolę nad przebiegiem treningu i tym, jaka część została już wykonana, użyjemy paska postępu wyświetlanego funkcją ProgressBar. Będzie on wyświetlał wyniki uzyskane na zbiorze uczącym podczas kolejnych epok.

In [ ]:
trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(model, metrics={"acc": Accuracy(), "loss": Loss(nn.NLLLoss())}, device=device)
ProgressBar(persist=True).attach(trainer, output_transform=lambda x: {"batch loss": x})

Przed rozpoczęciem treningu sieci należy zdefiniować liczbę epok. Po każdej epoce można wyliczyć metryki uzyskiwane na zbiorze walidacyjnym - jest to zbiór danych, które nie są podawane do sieci w celu uczenia, więc uzyskane na nich metryki pozwalają ocenić, czy sieć się uczy oraz czy się nie przeucza.

Żeby przeprowadzić walidację w pakiecie ignite trzeba wywołać evaluator, do którego podaje się dane walidacyjne. Ten evaluator musi być umieszczony w funkcji "podpiętej" pod trainer - poniżej przykład, jak to zrobić i spowodować, że po każdej epoce będą wyświetlane wyniki walidacji.

Po zakończonym treningu evaluator posłuży nam do wyliczenia metryk na zbiorze testowym.

In [ ]:
@trainer.on(Events.EPOCH_COMPLETED) #określamy, że walidacja ma być przeprowadzona po zakończeniu epoki
def log_validation_results(trainer):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}" #określamy, że dokładność i loss 
                                                                                #mają być wyświetlone z dokładością 
                                                                                #do 2 miejsc po przecinku
          .format(trainer.state.epoch, metrics['acc'], metrics['loss']))

#trainer.run(trainloader, max_epochs=1000)
trainer.run(train_loader, max_epochs=150)

evaluator.run(test_loader)
print(evaluator.state.metrics)

[1/33]   3%|3          [00:00<?]

Validation Results - Epoch: 1  Avg accuracy: 0.02 Avg loss: 5.78


[1/33]   3%|3          [00:00<?]

Validation Results - Epoch: 2  Avg accuracy: 0.02 Avg loss: 5.76


[1/33]   3%|3          [00:00<?]

...

Validation Results - Epoch: 8  Avg accuracy: 0.02 Avg loss: 5.62


[1/33]   3%|3          [00:00<?]

Validation Results - Epoch: 9  Avg accuracy: 0.02 Avg loss: 5.60


[1/33]   3%|3          [00:00<?]

Validation Results - Epoch: 10  Avg accuracy: 0.02 Avg loss: 5.58
{'acc': 0.022857142857142857, 'loss': 5.516507626488095}


Współczynnik uczenia może być dobierany w sposób automatyczny, służy do tego funkcja FastaiLRFinder. Dobrana wartość zależy od tego, ile wynosi parametr diverge_th - poszukiwania najlepszej wartośći współczynnika uczenia przerywane są w momencie, gdy zostanie spełnione kryterium current loss > diverge_th * best_loss.

diverge_th może być dowolną liczbą nie mniejszą niż 1, domyślnie wynosi 5. Mniejsza wartość powoduje szybsze zakończenie poszukiwań najlepszego współczynnika uczenia, jednak jest dość ryzykowne - jeżeli na początku poszukiwac trafimy na minimum lokalne, to szukanie zostanie zakończone. W praktyce może okazać się, że niezatrzymanie poszukiwań i zezwolenie na tymczasowe osiąganie większych wartości current_loss pozwoliłoby trafić na minimum globalne lub przynajmniej "lepsze" minimum lokalne, a tym samym lepiej dobrać wartość parametru i w konsekwencji przeprowadzić bardziej efektywny trening sieci.

In [ ]:
lr_finder = FastaiLRFinder()
to_save={'model': model, 'optimizer': optimizer}
with lr_finder.attach(trainer, to_save, diverge_th=1.05, start_lr=1e-8, end_lr=1e-7) as trainer_with_lr_finder:
    #domyślnie start_lr jest taki, jak określony w optimizerze, a end_lr=10
    trainer_with_lr_finder.run(train_loader)

print("Suggested LR", lr_finder.lr_suggestion())

[1/33]   3%|3          [00:00<?]

Validation Results - Epoch: 1  Avg accuracy: 0.03 Avg loss: 5.15
Suggested LR 2.154434690031884e-08


Po znalezieniu najlepszej wartości współczynnika uczenia trzeba ustawić tę wartość w optimizerze - robi się to funkcją apply_suggested_lr. Następnie trzeba ponownie przeprowadzić trening sieci.

Ponownie inicjalizujemy trainer - jeżeli tego nie zrobimy, to zamiast uczyć nową sieć neuronową będziemy kontynuować trening poprzedniej, co nie pozwoli nam dobrze przeanalizować wpływu parametru learning rate na proces uczenia.

In [ ]:
trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
# evaluator = create_supervised_evaluator(model, metrics={"acc": Accuracy(), "loss": Loss(nn.NLLLoss())}, device=device)
ProgressBar(persist=True).attach(trainer, output_transform=lambda x: {"batch loss": x})

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
          .format(trainer.state.epoch, metrics['acc'], metrics['loss']))

lr_finder.apply_suggested_lr(optimizer)
print('Training with suggested lr: ', optimizer.param_groups[0]['lr'])
#trainer.run(trainloader, max_epochs=1000)
trainer.run(train_loader, max_epochs=150)

evaluator.run(test_loader)
print(evaluator.state.metrics)

Training with suggested lr:  2.154434690031884e-08


[1/33]   3%|3          [00:00<?]

Validation Results - Epoch: 1  Avg accuracy: 0.02 Avg loss: 5.54


[1/33]   3%|3          [00:00<?]

Validation Results - Epoch: 2  Avg accuracy: 0.02 Avg loss: 5.50


...

Validation Results - Epoch: 8  Avg accuracy: 0.02 Avg loss: 5.27


[1/33]   3%|3          [00:00<?]

Validation Results - Epoch: 9  Avg accuracy: 0.02 Avg loss: 5.24


[1/33]   3%|3          [00:00<?]

Validation Results - Epoch: 10  Avg accuracy: 0.02 Avg loss: 5.21
{'acc': 0.021904761904761906, 'loss': 5.150043247767857}


Sieci neuronowe, zwłaszcza głębokie, mają niestety tendencję do przeuczania się  przy małej liczbie danych uczących. Nie zawsze jesteśmy w stanie powiększyć zbiór uczący o nowe dane, więc konieczne jest zastosowanie innych technik pozwalających zmniejszyć przeuczenie.

Jedną z takich technik jest dropout. Polega on na losowym "wyłączaniu" neuronów podczas kolejnych iteracji (czyli ustawianiu ich wag na 0) oraz skalowaniu niewyłączonych przez współczynnik 1/(1-p), gdzie p to zadany parametr dropoutu. Dzięki temu sieć nie może dopasować się idealnie do danych uczących, ponieważ podczas treningu musi dopasować wagi w taki sposób, by uzyskać dobre rezultaty również wtedy, gdy część neuronów będzie nieaktywna.

Definiując warstwę dropout należy określić, z jakim prawdopodobieństwem neurony będą "wyłączane". Domyślnie to prawdopodobieństwo wynosi 0.5 - można tę wartość zmienić, ale należy robić to w sposób przemyślany. Jeżeli ustawimy zbyt dużą wartość prawdopodobieństwa, to sieć może nie być w stanie w ogóle nauczyć się klasyfikować danych. Jeżeli ustawimy wartość zbyt małą, to regularyzacja może być nieefektywna i nie zmniejszymy przeuczenia sieci.

Jeżeli planujesz użyć dropoutów, zwłaszcza z różnymi wartościami p, to dobrze jest je zdefiniować w funkcji \_\_init\_\_ i potem odnosić się do nich po nazwie w funkcji forward, np.

def \_\_init\_\_(self):

    self.fc1 = nn.Linear(2574, 256)
    self.fc2 = nn.Linear(256, 120)
    self.fc3 = nn.Linear(120, 35)
    self.dropout1 = nn.Dropout(p=0.5)
    self.dropout2 = nn.Dropout(p=0.2)
    
def forward(self, x):

        x = self.fc1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

Spróbuj zmienić architekturę sieci tak, by uzyskać lepsze wyniki na zbiorze walidacyjnym i zbiorze testowym.

Możesz to zrobić:
- zwiększając liczbę warstw (uwaga na liczbę wejść i wyjść),
- dodając dropout,
- zmieniając funkcję aktywacji warstw (wszystkich lub niektórych), np.: x = F.relu(self.fc1(x))
- zmieniając optimizer.

Po zmianach zwróć uwagę, czy ustalona liczba epok wystarcza do nauczenia sieci - jeżeli do ostatniej epoki obserwujesz wzrost dokładności na zbiorze walidacyjnym, to prawdopodobnie sieć nadal nie jest w pełni nauczona i powinno się zwiększyć liczbę epok. Jeżeli od pewnego momentu treningu obserwujesz spadek dokładności, to sieć zaczęła się przeuczać i trzeba zmniejszyć liczbę epok (uwaga: musi to być trend spadkowy, a nie tylko pojedyncze spadki - one nic nie znaczą).